# Behavior Classification using Explainable Active Learning Model

## 1. Load Data

## 2. Preprocessing using A-SOiD Pipeline

### 2.1 Preprocessing parameters

In [ ]:
FRAME_RATE = 30
SAMPLE_RATE = 10
MULTI_ANIMAL = False

## 3.